### Recogida y envío de datos históricos de Consumo y Generación del Ed. SEGAI a ThingsBoard

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import glob
import pandas as pd
import requests
import json

download_folder = "C:\\Users\\angel\\Desktop\\MASTER\\TFM"

options = webdriver.ChromeOptions()  # Opciones de navegación con selenium
options.add_argument("--headless")  
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")
prefs = {
    "download.default_directory": download_folder,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
    "safebrowsing.disable_download_protection": True,
    "profile.default_content_setting_values.automatic_downloads": 1,
}
options.add_experimental_option("prefs", prefs)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Función para esperar la descarga completa del archivo y guardarlo como el archivo más reciente
def esperar_descarga_completa(extension="xls", timeout=60):
    start_time = time.time()
    while True:
        files = glob.glob(download_folder + f"/*.{extension}")
        if files:
            latest_file = max(files, key=os.path.getctime)  # Obtener el archivo más reciente
            print("Archivo descargado con éxito: " + latest_file)
            print()
            return latest_file
        elif time.time() - start_time > timeout:
            print("Tiempo de espera para la descarga excedido.")
            print()
            return None
        time.sleep(1)


# Función para enviar datos a ThingsBoard
def enviar_a_thingsboard(archivo_reciente):
    
    url = "https://greenenergy.iaas.ull.es:8080/api/v1/1s32b4n5k9bribml4yqs/telemetry"
    token = "1s32b4n5k9bribml4yqs"
    
    df = pd.read_excel(archivo_reciente, header=2)
    envios_exitosos = 0
    
    for index, row in df.iterrows():
        data = {
            "ts": pd.to_datetime(row['Time']).timestamp() * 1000,
            "values": {
                "PV(W)": row['PV(W)'],
                "Meter(W)": row['Meter(W)'],
                "Load(W)": row['Load(W)']
            }
        }
        headers = {'Content-Type': 'application/json', 'X-Authorization': f"Bearer {token}"}
        response = requests.post(url, data=json.dumps(data), headers=headers)
        if response.status_code == 200:
            print(f"Dato enviado correctamente: {data}")
            envios_exitosos += 1
        else:
            print(f"Error al enviar dato: {response.text}")
    
    if envios_exitosos == len(df):
        print("Todos los datos han sido enviados a ThingsBoard exitosamente. Eliminando archivo...")
        os.remove(archivo_reciente)
        print("Archivo eliminado con éxito.")
        print()
    else:
        print("Algunos datos no se pudieron enviar correctamente. El archivo no se eliminará.")
        print()
        
# Inicio de sesión y descarga de datos de Sems Portal para los últimos 351 días
login_url = "https://www.semsportal.com/home/login"
driver.get(login_url)

# Completar el formulario de inicio de sesión
time.sleep(2)  # Esperar a que la página cargue

# Esperar a que la página cargue y localizar la casilla de condiciones de uso y política de privacidad
checkbox = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "readStatement"))
)

# Hacer clic en la casilla de verificación si no está ya seleccionada
if not checkbox.is_selected():
    checkbox.click()
    
email_input = driver.find_element(By.ID, "username")
password_input = driver.find_element(By.ID, "password")
login_button = driver.find_element(By.ID, "btnLogin")

# Credenciales
your_email = "*****************"
your_password = "********"

email_input.send_keys(your_email)
password_input.send_keys(your_password)
login_button.click()

time.sleep(5)  # Esperar a que la página cargue

# Localizar el elemento por el texto que contiene
plant_link = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'SEGAI - ULL')]"))
)
plant_link.click()

# Esperar a que la página cargue después del inicio de sesión
time.sleep(5)  # Espera para que todos los elementos se carguen completamente

# Navegar a la URL específica
driver.get("https://www.semsportal.com/powerstation/PowerStatusSnMin/2c6eb65e-dca9-4c6f-810f-769064cc6ca8")

# No se retrocede un día antes de la primera descarga
for day in range(370):  # Incluye el día actual y los 350 días anteriores
    max_retries = 3
    archivo_descargado = False  # Indica si el archivo ha sido descargado correctamente

    for attempt in range(max_retries):
        try:
            WebDriverWait(driver, 20).until(EC.invisibility_of_element((By.CSS_SELECTOR, '.el-loading-mask')))
            WebDriverWait(driver, 20).until(EC.invisibility_of_element((By.CSS_SELECTOR, '.el-loading-spinner')))
            
            action = ActionChains(driver)
            export_icon = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".goodwe-station-charts__export.fr"))
            )
            action.move_to_element(export_icon).click().perform()

            archivo_reciente = esperar_descarga_completa(extension="xls")
            if archivo_reciente:
                print(f"Clic en el icono de exportación realizado con éxito y archivo descargado para el día actual o el día {day} en retrospectiva.")
                print()
                enviar_a_thingsboard(archivo_reciente)
                archivo_descargado = True
                break  # Sale del bucle de reintentos si se descarga el archivo con éxito
            else:
                print(f"Intento {attempt + 1} de descarga fallido. Reintentando...")
                print()
        except Exception as e:
            print(f"Ocurrió un error durante el intento {attempt + 1}: {e}")
            print()

        time.sleep(5)  # Espera antes de reintentar

    # Si el archivo se descargó correctamente o después del primer día, procede a retroceder un día para la siguiente iteración
    if day != 0 and archivo_descargado:  # Se añade la comprobación de archivo_descargado para asegurar que solo retrocedemos si el día actual fue exitoso
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".station-date-picker_left"))).click()
        time.sleep(2)  # Espera a que la página se actualice

# Cerrar el navegador al finalizar todas las descargas
driver.quit()

# Una vez descargado se espera a que la descarga finalice
archivo_reciente = esperar_descarga_completa(extension="xls")

# Si el archivo se descargó correctamente se procede a enviar los datos a ThingsBoard
if archivo_reciente:
    enviar_a_thingsboard(archivo_reciente)

Archivo descargado con éxito: C:\Users\angel\Desktop\MASTER\TFM\Plant Power_20240404053859.xls

Clic en el icono de exportación realizado con éxito y archivo descargado para el día actual o el día 0 en retrospectiva.

Dato enviado correctamente: {'ts': 1712102400000.0, 'values': {'PV(W)': 0, 'Meter(W)': -35297, 'Load(W)': 35297}}
Dato enviado correctamente: {'ts': 1712102700000.0, 'values': {'PV(W)': 0, 'Meter(W)': -30583, 'Load(W)': 30583}}
Dato enviado correctamente: {'ts': 1712103000000.0, 'values': {'PV(W)': 0, 'Meter(W)': -29802, 'Load(W)': 29802}}
Dato enviado correctamente: {'ts': 1712103300000.0, 'values': {'PV(W)': 0, 'Meter(W)': -30172, 'Load(W)': 30172}}
Dato enviado correctamente: {'ts': 1712103600000.0, 'values': {'PV(W)': 0, 'Meter(W)': -53740, 'Load(W)': 53740}}
Dato enviado correctamente: {'ts': 1712103900000.0, 'values': {'PV(W)': 0, 'Meter(W)': -54272, 'Load(W)': 54272}}
Dato enviado correctamente: {'ts': 1712104200000.0, 'values': {'PV(W)': 0, 'Meter(W)': -29646, 'Lo

Dato enviado correctamente: {'ts': 1712123400000.0, 'values': {'PV(W)': 0, 'Meter(W)': -28527, 'Load(W)': 28527}}
Dato enviado correctamente: {'ts': 1712123700000.0, 'values': {'PV(W)': 0, 'Meter(W)': -28567, 'Load(W)': 28567}}
Dato enviado correctamente: {'ts': 1712124000000.0, 'values': {'PV(W)': 0, 'Meter(W)': -30023, 'Load(W)': 30023}}
Dato enviado correctamente: {'ts': 1712124300000.0, 'values': {'PV(W)': 0, 'Meter(W)': -30563, 'Load(W)': 30563}}
Dato enviado correctamente: {'ts': 1712124600000.0, 'values': {'PV(W)': 0, 'Meter(W)': -30653, 'Load(W)': 30653}}
Dato enviado correctamente: {'ts': 1712124900000.0, 'values': {'PV(W)': 0, 'Meter(W)': -33081, 'Load(W)': 33081}}
Dato enviado correctamente: {'ts': 1712125200000.0, 'values': {'PV(W)': 0, 'Meter(W)': -53738, 'Load(W)': 53738}}
Dato enviado correctamente: {'ts': 1712125500000.0, 'values': {'PV(W)': 0, 'Meter(W)': -54140, 'Load(W)': 54140}}
Dato enviado correctamente: {'ts': 1712125800000.0, 'values': {'PV(W)': 0, 'Meter(W)': -

Dato enviado correctamente: {'ts': 1712144700000.0, 'values': {'PV(W)': 46707, 'Meter(W)': 6618, 'Load(W)': 40089}}
Dato enviado correctamente: {'ts': 1712145000000.0, 'values': {'PV(W)': 45088, 'Meter(W)': 534, 'Load(W)': 44554}}
Dato enviado correctamente: {'ts': 1712145300000.0, 'values': {'PV(W)': 47824, 'Meter(W)': 5779, 'Load(W)': 42045}}
Dato enviado correctamente: {'ts': 1712145600000.0, 'values': {'PV(W)': 47421, 'Meter(W)': 5511, 'Load(W)': 41910}}
Dato enviado correctamente: {'ts': 1712145900000.0, 'values': {'PV(W)': 47312, 'Meter(W)': -14026, 'Load(W)': 61338}}
Dato enviado correctamente: {'ts': 1712146200000.0, 'values': {'PV(W)': 49746, 'Meter(W)': -12036, 'Load(W)': 61782}}
Dato enviado correctamente: {'ts': 1712146500000.0, 'values': {'PV(W)': 50263, 'Meter(W)': -16056, 'Load(W)': 66319}}
Dato enviado correctamente: {'ts': 1712146800000.0, 'values': {'PV(W)': 50917, 'Meter(W)': -15608, 'Load(W)': 66525}}
Dato enviado correctamente: {'ts': 1712147100000.0, 'values': {'P

Dato enviado correctamente: {'ts': 1712166000000.0, 'values': {'PV(W)': 34919, 'Meter(W)': 4900, 'Load(W)': 30019}}
Dato enviado correctamente: {'ts': 1712166300000.0, 'values': {'PV(W)': 33637, 'Meter(W)': 1009, 'Load(W)': 32628}}
Dato enviado correctamente: {'ts': 1712166600000.0, 'values': {'PV(W)': 33043, 'Meter(W)': 1784, 'Load(W)': 31259}}
Dato enviado correctamente: {'ts': 1712166900000.0, 'values': {'PV(W)': 31170, 'Meter(W)': 533, 'Load(W)': 30637}}
Dato enviado correctamente: {'ts': 1712167200000.0, 'values': {'PV(W)': 29863, 'Meter(W)': -1283, 'Load(W)': 31146}}
Dato enviado correctamente: {'ts': 1712167500000.0, 'values': {'PV(W)': 28798, 'Meter(W)': -1745, 'Load(W)': 30543}}
Dato enviado correctamente: {'ts': 1712167800000.0, 'values': {'PV(W)': 26876, 'Meter(W)': -4374, 'Load(W)': 31250}}
Dato enviado correctamente: {'ts': 1712168100000.0, 'values': {'PV(W)': 25570, 'Meter(W)': -4528, 'Load(W)': 30098}}
Dato enviado correctamente: {'ts': 1712168400000.0, 'values': {'PV(W)

Dato enviado correctamente: {'ts': 1712105100000.0, 'values': {'PV(W)': 0, 'Meter(W)': -33148, 'Load(W)': 33148}}
Dato enviado correctamente: {'ts': 1712105400000.0, 'values': {'PV(W)': 0, 'Meter(W)': -32106, 'Load(W)': 32106}}
Dato enviado correctamente: {'ts': 1712105700000.0, 'values': {'PV(W)': 0, 'Meter(W)': -30291, 'Load(W)': 30291}}
Dato enviado correctamente: {'ts': 1712106000000.0, 'values': {'PV(W)': 0, 'Meter(W)': -27923, 'Load(W)': 27923}}
Dato enviado correctamente: {'ts': 1712106300000.0, 'values': {'PV(W)': 0, 'Meter(W)': -30711, 'Load(W)': 30711}}
Dato enviado correctamente: {'ts': 1712106600000.0, 'values': {'PV(W)': 0, 'Meter(W)': -29086, 'Load(W)': 29086}}
Dato enviado correctamente: {'ts': 1712106900000.0, 'values': {'PV(W)': 0, 'Meter(W)': -28804, 'Load(W)': 28804}}
Dato enviado correctamente: {'ts': 1712107200000.0, 'values': {'PV(W)': 0, 'Meter(W)': -30299, 'Load(W)': 30299}}
Dato enviado correctamente: {'ts': 1712107500000.0, 'values': {'PV(W)': 0, 'Meter(W)': -

...

Dato enviado correctamente: {'ts': 1712021700000.0, 'values': {'PV(W)': 0, 'Meter(W)': -30882, 'Load(W)': 30882}}
Dato enviado correctamente: {'ts': 1712022000000.0, 'values': {'PV(W)': 0, 'Meter(W)': -29729, 'Load(W)': 29729}}
Dato enviado correctamente: {'ts': 1712022300000.0, 'values': {'PV(W)': 0, 'Meter(W)': -31203, 'Load(W)': 31203}}
Dato enviado correctamente: {'ts': 1712022600000.0, 'values': {'PV(W)': 0, 'Meter(W)': -30276, 'Load(W)': 30276}}
Dato enviado correctamente: {'ts': 1712022900000.0, 'values': {'PV(W)': 0, 'Meter(W)': -29633, 'Load(W)': 29633}}
Dato enviado correctamente: {'ts': 1712023200000.0, 'values': {'PV(W)': 0, 'Meter(W)': -31270, 'Load(W)': 31270}}
Dato enviado correctamente: {'ts': 1712023500000.0, 'values': {'PV(W)': 0, 'Meter(W)': -28777, 'Load(W)': 28777}}
Dato enviado correctamente: {'ts': 1712023800000.0, 'values': {'PV(W)': 0, 'Meter(W)': -27632, 'Load(W)': 27632}}
Dato enviado correctamente: {'ts': 1712024100000.0, 'values': {'PV(W)': 0, 'Meter(W)': -

Dato enviado correctamente: {'ts': 1712043300000.0, 'values': {'PV(W)': 0, 'Meter(W)': -59051, 'Load(W)': 59051}}
Dato enviado correctamente: {'ts': 1712043600000.0, 'values': {'PV(W)': 0, 'Meter(W)': -59722, 'Load(W)': 59722}}
Dato enviado correctamente: {'ts': 1712043900000.0, 'values': {'PV(W)': 0, 'Meter(W)': -35049, 'Load(W)': 35049}}
Dato enviado correctamente: {'ts': 1712044200000.0, 'values': {'PV(W)': 0, 'Meter(W)': -35306, 'Load(W)': 35306}}
Dato enviado correctamente: {'ts': 1712044500000.0, 'values': {'PV(W)': 0, 'Meter(W)': -44112, 'Load(W)': 44112}}
Dato enviado correctamente: {'ts': 1712044800000.0, 'values': {'PV(W)': 0, 'Meter(W)': -36100, 'Load(W)': 36100}}
Dato enviado correctamente: {'ts': 1712045100000.0, 'values': {'PV(W)': 306, 'Meter(W)': -35887, 'Load(W)': 36193}}
Dato enviado correctamente: {'ts': 1712045400000.0, 'values': {'PV(W)': 482, 'Meter(W)': -36026, 'Load(W)': 36508}}
Dato enviado correctamente: {'ts': 1712045700000.0, 'values': {'PV(W)': 708, 'Meter(

Dato enviado correctamente: {'ts': 1712064600000.0, 'values': {'PV(W)': 40190, 'Meter(W)': -6175, 'Load(W)': 46365}}
Dato enviado correctamente: {'ts': 1712064900000.0, 'values': {'PV(W)': 17826, 'Meter(W)': -47159, 'Load(W)': 64985}}
Dato enviado correctamente: {'ts': 1712065200000.0, 'values': {'PV(W)': 21054, 'Meter(W)': -22476, 'Load(W)': 43530}}
Dato enviado correctamente: {'ts': 1712065500000.0, 'values': {'PV(W)': 32116, 'Meter(W)': 12456, 'Load(W)': 19660}}
Dato enviado correctamente: {'ts': 1712065800000.0, 'values': {'PV(W)': 27742, 'Meter(W)': -11456, 'Load(W)': 39198}}
Dato enviado correctamente: {'ts': 1712066100000.0, 'values': {'PV(W)': 21385, 'Meter(W)': -15722, 'Load(W)': 37107}}
Dato enviado correctamente: {'ts': 1712066400000.0, 'values': {'PV(W)': 32327, 'Meter(W)': -4420, 'Load(W)': 36747}}
Dato enviado correctamente: {'ts': 1712066700000.0, 'values': {'PV(W)': 34357, 'Meter(W)': -6828, 'Load(W)': 41185}}
Dato enviado correctamente: {'ts': 1712067000000.0, 'values'

Dato enviado correctamente: {'ts': 1712085900000.0, 'values': {'PV(W)': 6247, 'Meter(W)': -28584, 'Load(W)': 34831}}
Dato enviado correctamente: {'ts': 1712086200000.0, 'values': {'PV(W)': 3859, 'Meter(W)': -30739, 'Load(W)': 34598}}
Dato enviado correctamente: {'ts': 1712086500000.0, 'values': {'PV(W)': 2451, 'Meter(W)': -32782, 'Load(W)': 35233}}
Dato enviado correctamente: {'ts': 1712086800000.0, 'values': {'PV(W)': 1272, 'Meter(W)': -34781, 'Load(W)': 36053}}
Dato enviado correctamente: {'ts': 1712087100000.0, 'values': {'PV(W)': 910, 'Meter(W)': -36659, 'Load(W)': 37569}}
Dato enviado correctamente: {'ts': 1712087400000.0, 'values': {'PV(W)': 883, 'Meter(W)': -34651, 'Load(W)': 35534}}
Dato enviado correctamente: {'ts': 1712087700000.0, 'values': {'PV(W)': 735, 'Meter(W)': -34312, 'Load(W)': 35047}}
Dato enviado correctamente: {'ts': 1712088000000.0, 'values': {'PV(W)': 467, 'Meter(W)': -33539, 'Load(W)': 34006}}
Dato enviado correctamente: {'ts': 1712088300000.0, 'values': {'PV(W

KeyboardInterrupt: 